# HPT: PyTorch With VBDP {#sec-vbdp}

In this tutorial, we will show how `spotPython` can be integrated into the `PyTorch`
training workflow for a classifiaction task.

::: {.callout-caution}
### Caution: Data must be downloaded manually

* Ensure that the correspondiing data is available as `./data/VBDP/train.csv`.

:::

This document refers to the following software versions:

- ``python``: 3.10.10
- ``torch``: 2.0.1
- ``torchvision``: 0.15.0


In [ ]:
pip list | grep  "spot[RiverPython]"

`spotPython` can be installed via pip. Alternatively, the source code can be downloaded from gitHub: [https://github.com/sequential-parameter-optimization/spotPython](https://github.com/sequential-parameter-optimization/spotPython).

```{raw}
!pip install spotPython
```

* Uncomment the following lines if you want to for (re-)installation the latest version of `spotPython` from gitHub.

In [ ]:
# import sys
# !{sys.executable} -m pip install --upgrade build
# !{sys.executable} -m pip install --upgrade --force-reinstall spotPython

## Step 1: Setup {#sec-setup-25}

Before we consider the detailed experimental setup, we select the parameters that affect run time, initial design size and the device that is used.

::: {.callout-caution}
### Caution: Run time and initial design size should be increased for real experiments

* MAX_TIME is set to one minute for demonstration purposes. For real experiments, this should be increased to at least 1 hour.
* INIT_SIZE is set to 5 for demonstration purposes. For real experiments, this should be increased to at least 10.

:::

::: {.callout-note}
### Note: Device selection

* The device can be selected by setting the variable `DEVICE`.
* Since we are using a simple neural net, the setting `"cpu"` is preferred (on Mac).
* If you have a GPU, you can use `"cuda:0"` instead.
* If DEVICE is set to `None`, `spotPython` will automatically select the device.
  * This might result in `"mps"` on Macs, which is not the best choice for simple neural nets.

:::


In [ ]:
MAX_TIME = 1
INIT_SIZE = 5
DEVICE = None, #"cpu" # "cuda:0"

In [ ]:
from spotPython.utils.device import getDevice
DEVICE = getDevice(DEVICE)
print(DEVICE)

In [ ]:
import os
import copy
import socket
from datetime import datetime
from dateutil.tz import tzlocal
start_time = datetime.now(tzlocal())
HOSTNAME = socket.gethostname().split(".")[0]
experiment_name = '30-light' + "_" + HOSTNAME + "_" + str(MAX_TIME) + "min_" + str(INIT_SIZE) + "init_" + str(start_time).split(".", 1)[0].replace(' ', '_')
experiment_name = experiment_name.replace(':', '-')
print(experiment_name)
if not os.path.exists('./figures'):
    os.makedirs('./figures')

## Step 2: Initialization of the `fun_control` Dictionary

:::{.callout-caution}
### Caution: Tensorboard does not work under Windows
* Since tensorboard does not work under Windows, we recommend setting the parameter `tensorboard_path` to `None` if you are working under Windows.
:::

`spotPython` uses a Python dictionary for storing the information required for the hyperparameter tuning process, which was described in @sec-initialization-fun-control-14, see [Initialization of the fun_control Dictionary](https://sequential-parameter-optimization.github.io/spotPython/14_spot_ray_hpt_torch_cifar10.html#sec-initialization-fun-control-14) in the documentation.


In [ ]:
from spotPython.utils.init import fun_control_init
fun_control = fun_control_init(task="classification",
    tensorboard_path="./runs/" + experiment_name,
    num_workers=10,
    device=DEVICE)

## Step 3: PyTorch Data Loading {#sec-data-loading-25}

### 1. Load VBDP Data

In [ ]:
import torch
from spotPython.light.csvdataset import CSVDataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

# Create an instance of CSVDataset
dataset = CSVDataset(csv_file="./data/VBDP/train.csv", train=True)
# show the dimensions of the dataset
print(dataset[0][0].shape)
# show the first element of the dataset
print(dataset[0][0])

In [ ]:
# Create an instance of CSVDataset
dataset = CSVDataset(csv_file="./data/VBDP/train.csv", train=False)
# show the size of the dataset
print(f"Dataset Size: {len(dataset)}")

In [ ]:

# Set batch size for DataLoader
batch_size = 3
# Create DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Iterate over the data in the DataLoader
for batch in dataloader:
    inputs, targets = batch
    print(f"Batch Size: {inputs.size(0)}")
    print("---------------")
    print(f"Inputs: {inputs}")
    print(f"Targets: {targets}")
    break


In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import Subset, DataLoader

dataset_full = CSVDataset(csv_file="./data/VBDP/train.csv", train=False)
kf = KFold(n_splits=10, shuffle=True, random_state=123)
all_splits = [k for k in kf.split(dataset_full)]
train_indexes, val_indexes = all_splits[8]
train_indexes, val_indexes = train_indexes.tolist(), val_indexes.tolist()

# Create subsets using Subset class
train_subset = Subset(dataset_full, train_indexes)
val_subset = Subset(dataset_full, val_indexes)

# Create DataLoaders for training and validation subsets
train_dataloader = DataLoader(train_subset, batch_size=3, shuffle=True)
val_dataloader = DataLoader(val_subset, batch_size=3, shuffle=True)

# Iterate over the data in the validation DataLoader
for batch in val_dataloader:
    inputs, targets = batch
    print(f"Batch Size: {inputs.size(0)}")
    print("---------------")
    print(f"Inputs: {inputs}")
    print(f"Targets: {targets}")
    break


## Step 4: Specification of the Preprocessing Model {#sec-specification-of-preprocessing-model-25}

After the training and test data are specified and added to the `fun_control` dictionary, `spotPython` allows the specification of a data preprocessing pipeline, e.g., for the scaling of the data or for the one-hot encoding of categorical variables, see @sec-specification-of-preprocessing-model-14. This feature is not used here, so we do not change the default value (which is `None`).

:::{.callout-note}
Lightning allows the data preprocessing to be specified in the `LightningDataModule` class. It is not considered here, because it should be computed at one location only.
:::


## Step 5: Select `algorithm` and `core_model_hyper_dict` {#sec-selection-of-the-algorithm-25}

### Implementing a Configurable Neural Network With spotPython 

`spotPython` includes the `Net_vbdp` class which is implemented in the file `netvbdp.py`.
The class is imported here.

This class  inherits from the class `Net_Core` which is implemented in the file `netcore.py`, see @sec-the-netcore-class-14.

### Add the NN Model to the fun_control Dictionary

In [ ]:
from spotPython.light.csvmodel import CSVModel 
from spotPython.data.light_hyper_dict import LightHyperDict
from spotPython.hyperparameters.values import add_core_model_to_fun_control
fun_control = add_core_model_to_fun_control(core_model=CSVModel,
                              fun_control=fun_control,
                              hyper_dict= LightHyperDict)

The corresponding entries for the `core_model` class are shown below.

In [ ]:
fun_control['core_model_hyper_dict']

## Step 6: Modify `hyper_dict` Hyperparameters for the Selected Algorithm aka `core_model` {#sec-modification-of-hyperparameters-25}

 `spotPython` provides functions for modifying the hyperparameters, their bounds and factors as well as for activating and de-activating hyperparameters without re-compilation of the Python source code. These functions were described in @sec-modification-of-hyperparameters-14.

::: {.callout-caution}
### Caution: Small number of epochs for demonstration purposes

* `epochs` and `patience` are set to small values for demonstration purposes. These values are too small for a real application.
* More resonable values are, e.g.:
  * `fun_control = modify_hyper_parameter_bounds(fun_control, "epochs", bounds=[7, 9])` and
  * `fun_control = modify_hyper_parameter_bounds(fun_control, "patience", bounds=[2, 7])`
:::

In [ ]:
from spotPython.hyperparameters.values import modify_hyper_parameter_bounds

fun_control = modify_hyper_parameter_bounds(fun_control, "l1", bounds=[2,3])
fun_control = modify_hyper_parameter_bounds(fun_control, "epochs", bounds=[1,2])
fun_control = modify_hyper_parameter_bounds(fun_control, "batch_size", bounds=[6, 8])

In [ ]:
from spotPython.hyperparameters.values import modify_hyper_parameter_levels
fun_control = modify_hyper_parameter_levels(fun_control, "optimizer",["Adam", "AdamW", "Adamax", "NAdam"])
# fun_control = modify_hyper_parameter_levels(fun_control, "optimizer", ["Adam"])
# fun_control["core_model_hyper_dict"]

## Step 7: Selection of the Objective (Loss) Function

### Evaluation  {#sec-selection-of-target-function-25}

The evaluation procedure requires the specification of two elements:

1. the way how the data is split into a train and a test set (see @sec-data-splitting-14)
2. the loss function (and a metric).


### Loss Functions and Metrics {#sec-loss-functions-and-metrics-25}

The loss function is specified by the key `"loss_function"`.
We will use CrossEntropy loss for the multiclass-classification task.

In [ ]:
# from torch.nn import CrossEntropyLoss
# loss_function = CrossEntropyLoss()
# fun_control.update({"loss_function": loss_function})

### Metric {#sec-metric-25}

* We will use the MAP@k metric for the evaluation of the model. Here is an example how this metric is calculated.

In [ ]:
from spotPython.torch.mapk import MAPK
import torch
mapk = MAPK(k=2)
target = torch.tensor([0, 1, 2, 2])
preds = torch.tensor(
    [
        [0.5, 0.2, 0.2],  # 0 is in top 2
        [0.3, 0.4, 0.2],  # 1 is in top 2
        [0.2, 0.4, 0.3],  # 2 is in top 2
        [0.7, 0.2, 0.1],  # 2 isn't in top 2
    ]
)
mapk.update(preds, target)
print(mapk.compute()) # tensor(0.6250)

In [ ]:
# from spotPython.torch.mapk import MAPK
# import torchmetrics
# metric_torch = MAPK(k=3)
# fun_control.update({"metric_torch": metric_torch})

## Step 8: Calling the SPOT Function

### Preparing the SPOT Call {#sec-prepare-spot-call-25}

The following code passes the information about the parameter ranges and bounds to `spot`.

In [ ]:
# extract the variable types, names, and bounds
from spotPython.hyperparameters.values import (get_bound_values,
    get_var_name,
    get_var_type,)
var_type = get_var_type(fun_control)
var_name = get_var_name(fun_control)
fun_control.update({"var_type": var_type,
                    "var_name": var_name})
lower = get_bound_values(fun_control, "lower")
upper = get_bound_values(fun_control, "upper")

Now, the dictionary `fun_control` contains all information needed for the hyperparameter tuning. Before the hyperparameter tuning is started, it is recommended to take a look at the experimental design. The method `gen_design_table` generates a design table as follows:

In [ ]:
#| fig-cap: Experimental design for the hyperparameter tuning.
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control))

This allows to check if all information is available and if the information is correct.

### The Objective Function `fun_torch` {#sec-the-objective-function-25}

The objective function `fun_torch` is selected next. It implements an interface from `PyTorch`'s training, validation, and  testing methods to `spotPython`.


In [ ]:
from spotPython.light.hyperlight import HyperLight
fun = HyperLight().fun

In [ ]:
from spotPython.hyperparameters.values import get_default_hyperparameters_as_array
hyper_dict=LightHyperDict().load()
X_start = get_default_hyperparameters_as_array(fun_control, hyper_dict)
X_start

### Starting the Hyperparameter Tuning {#sec-call-the-hyperparameter-tuner-25}

The `spotPython` hyperparameter tuning is started by calling the `Spot` function as described in @sec-call-the-hyperparameter-tuner-14.


In [ ]:
import numpy as np
from spotPython.spot import spot
from math import inf
spot_tuner = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   fun_repeats = 1,
                   max_time = MAX_TIME,
                   noise = False,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type = var_type,
                   var_name = var_name,
                   infill_criterion = "y",
                   n_points = 1,
                   seed=123,
                   log_level = 50,
                   show_models= False,
                   show_progress= True,
                   fun_control = fun_control,
                   design_control={"init_size": INIT_SIZE,
                                   "repeats": 1},
                   surrogate_control={"noise": True,
                                      "cod_type": "norm",
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": len(var_name),
                                      "model_fun_evals": 10_000,
                                      "log_level": 50
                                      })
spot_tuner.run(X_start=X_start)

## Step 9: Tensorboard {#sec-tensorboard-25}

The textual output shown in the console (or code cell) can be visualized with Tensorboard as described in @sec-tensorboard-14, see also the description in the documentation: [Tensorboard.](https://sequential-parameter-optimization.github.io/spotPython/14_spot_ray_hpt_torch_cifar10.html#sec-tensorboard-14)

## Step 10: Results {#sec-results-25}

After the hyperparameter tuning run is finished, the results can be analyzed as described in @sec-results-14.

In [ ]:
#| fig-cap: Progress plot. *Black* dots denote results from the initial design. *Red* dots  illustrate the improvement found by the surrogate model based optimization.
spot_tuner.plot_progress(log_y=False, 
    filename="./figures/" + experiment_name+"_progress.png")

In [ ]:
#| fig-cap: Results of the hyperparameter tuning.
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

In [ ]:
#| fig-cap: 'Variable importance plot, threshold 0.025.'
spot_tuner.plot_importance(threshold=0.025,
    filename="./figures/" + experiment_name+"_importance.png")

### Get the Tuned Architecture {#sec-get-spot-results-25}

In [ ]:
from spotPython.hyperparameters.values import get_one_config_from_X
X = spot_tuner.to_all_dim(spot_tuner.min_X.reshape(1,-1))
config = get_one_config_from_X(X, fun_control)
config

* Test on the full data set

In [ ]:
from spotPython.light.traintest import test_model
test_model(config, fun_control)

### Cross Validation With Lightning

* This is the evaluation that will be used in the comparison.

In [ ]:
from spotPython.light.traintest import cv_model
cv_model(config, fun_control)

### Detailed Hyperparameter Plots

In [ ]:
#| fig-cap: Contour plots.
filename = "./figures/" + experiment_name
spot_tuner.plot_important_hyperparameter_contour(filename=filename)

### Parallel Coordinates Plot

In [ ]:
#| fig-cap: Parallel coordinates plots
spot_tuner.parallel_plot()

### Plot all Combinations of Hyperparameters

* Warning: this may take a while.

In [ ]:
PLOT_ALL = False
if PLOT_ALL:
    n = spot_tuner.k
    for i in range(n-1):
        for j in range(i+1, n):
            spot_tuner.plot_contour(i=i, j=j, min_z=min_z, max_z = max_z)